In [81]:
# import libraries
import pandas as pd
import numpy as np


Get the data from Wikipedia with pandas


In [82]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
list_df = pd.read_html(url)

The created list of Data Frames contain 3 data frames. The first one is the one we need. Let's assign it to a new data frame.



In [83]:
df = list_df[0]
print(df.shape)
# Display the first 10 rows
df.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Let's rename the first column to PostalCode



In [84]:
df.rename(columns={"Postal Code":"PostalCode"},inplace=True)

# Get rid of rows with no Borough
to_drop = np.where(df['Borough']=='Not assigned')[0]
df.drop(to_drop,axis=0,inplace=True)
df.shape

(103, 3)

Let's check if any "Not assigned" values are still in Borough...

In [85]:
(df['Borough']=='Not assigned').sum()

0

Let's reset the index and check how the new DF looks like...

In [86]:
df = df.reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Let's check now for "Not assigned" values in Neighbourhood

In [87]:
(df['Neighbourhood']=="Not assigned").sum()

0

Now we combine rows that have the same PostalCode value

In [88]:
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [89]:
df.shape

(103, 3)

# ==== End of question 1 ====